In [1]:
from cafecosmos import World, Player

abi_dir = "./cafecosmos-contracts/abi"
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"
indexer_url = "https://indexer.mud.garnetchain.com/q"
mud_config_path = "./cafecosmos-contracts/mud.config.ts"
block_explorer_url = "https://explorer.garnetchain.com"


cafecosmos = World(rpc, 
              world_address, 
              abi_dir, 
              indexer_url=indexer_url, 
              mud_config_path=mud_config_path,
              block_explorer_url=block_explorer_url)

player = Player(env_key_name="PLAYER1", world=cafecosmos, land_id=5)

/Users/nico/.virtualenvs/woww/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
player.place_item(4,0,"bismuth")

Transaction sent: https://explorer.garnetchain.com/tx/0x43d2b3ab12c20dc82e72953358b71e70800083a11acaf7d2bf2f762c0ff8503d
Transaction confirmed in block: 12159699


In [ ]:
# player.cafecosmos.indexer.LandItem.get(landId=player.land_id, x=4, y=0)


print(unlockable_transformations)
print(player_land)


[{'base': '20', 'input': '0', 'next': '0', 'yield': '20', 'inputnext': '0', 'yieldquantity': '24', 'unlocktime': '0', 'timeout': '0', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '0', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '23', 'input': '0', 'next': '0', 'yield': '23', 'inputnext': '0', 'yieldquantity': '8', 'unlocktime': '0', 'timeout': '0', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '0', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '41', 'input': '0', 'next': '0', 'yield': '129', 'inputnext': '0', 'yieldquantity': '1', 'unlocktime': '0', 'timeout': '360', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '131', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '47', 'input': '0', 'next': '0', 'yield': '14', 'inputnext': '0', 'yieldquantity': '3', 'unlocktime': '0', 'timeout': '360', 'timeoutyield': '0', 'timeou

In [32]:
import time

In [48]:
import time  # Make sure to import the time module

def get_unlockable_transformations(player: Player):

    unlockable_transformations = player.cafecosmos.indexer.Transformations.get(input=0)
    player_land = player.cafecosmos.indexer.LandItem.get(landId=player.land_id)
    timestamp = player.cafecosmos.w3.eth.get_block('latest').timestamp

    coordinates = []
    closest_unlock_time = None

    for item in player_land:
        # Extract the item ID from the player's land item
        item_id = str(item["itemid"])  # Ensure item_id is a string to match the data type in unlockable_transformations

        # Find the matching transformation in unlockable_transformations
        matching_transformation = next(
            (transformation for transformation in unlockable_transformations if transformation.get('base') == item_id),
            None
        )

        if matching_transformation:
            # Retrieve the unlocktime and calculate based on placementtime
            unlocktime = int(matching_transformation.get('unlocktime', 0))
            placementtime = int(item.get('placementtime', 0))
            total_unlock_time = unlocktime + placementtime

            # Check if the unlock time has passed
            is_unlockable = total_unlock_time < timestamp

            if not is_unlockable:
                if closest_unlock_time is None or total_unlock_time < closest_unlock_time:
                    closest_unlock_time = total_unlock_time

            coordinates.append((int(item['x']), int(item['y'])))
        
    return {
        "coordinates": coordinates,
        "nextUnlock": closest_unlock_time
    }


get_unlockable_transformations(player)

{'coordinates': [(4, 0), (6, 6), (9, 1)], 'nextUnlock': None}

In [ ]:
def unlock_all(player: Player):
    coordinates = get_unlockable_transformations(player)['coordinates']
    for coord in coordinates:
        player.place_item(coord[0],coord[1])

In [6]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [4]:
import time

print(time.time())
print(player.cafecosmos.w3.eth.get_block('latest').timestamp)

1736488356.0416021
1736488356


In [4]:
import pandas as pd

def get_unlockable_transformations(world: World, land_id: int) -> pd.DataFrame:
    """
    Determine which transformations can be unlocked based on the player's land inventory.

    Args:
        world (World): The world object containing indexers.
        land_id (int): The ID of the player's land.

    Returns:
        pd.DataFrame: A DataFrame of transformations that can be unlocked, including time until unlock.
    """
    # Fetch land items
    land_items = world.indexer.LandItem.get(landId=land_id)
    transformations = world.indexer.Transformations.get()
    timestamp = world.w3.eth.get_block('latest').timestamp

    # Filter transformations with 'input' == '0'
    filtered_results = [entry for entry in transformations if entry.get('input') == '0']

    print(filtered_results)

    # Prepare list for filtered items
    unlockable_items = []

    for item in land_items:
        item_id = int(item["itemid"])
        placement_time = int(item["placementtime"])
        
        if item_id == 0:  # Ignore placeholder or empty items
            continue

        # Check if this item_id exists in the filtered_results as a "base"
        if any(entry.get('base') == item_id for entry in filtered_results):
            # Only include items where placementtime is greater than the timestamp
            if placement_time > timestamp:
                time_until_unlock = placement_time - timestamp
                unlockable_items.append({
                    "item_id": item_id,
                    "x": int(item["x"]),
                    "y": int(item["y"]),
                    "z": int(item["z"]),
                    "placement_time": placement_time,
                    "time_until_unlock": time_until_unlock
                })

    # Convert to DataFrame for easier manipulation
    unlockable_items_df = pd.DataFrame(unlockable_items)

    print("Unlockable Items:")
    print(unlockable_items_df)

    # If no unlockable items exist, return empty DataFrame
    if unlockable_items_df.empty:
        print("No unlockable items found.")
        return unlockable_items_df

    # Find the minimum time until unlock
    next_unlock_time = unlockable_items_df["time_until_unlock"].min()
    print(f"Time until the next unlock: {next_unlock_time} seconds")

    return unlockable_items_df, next_unlock_time

    # # print("land_inventory", land_inventory)
    # # Determine unlockable transformations
    # unlockable = []
    # for trans in transformations:
    #     input_id = int(trans["input"])
    #     required_quantity = int(trans.get("yieldquantity", 1))  # Default to 1 if not specified
    #     if input_id in land_inventory and land_inventory[input_id] >= required_quantity:
    #         unlockable.append({
    #             "Base": int(trans["base"]),
    #             "Input": input_id,
    #             "Next": int(trans["next"]),
    #             "Yield": int(trans["yield"]),
    #             "YieldQuantity": required_quantity,
    #             "UnlockTime": int(trans["unlocktime"]),
    #             "Timeout": int(trans["timeout"]),
    #             "IsRecipe": trans["isrecipe"],
    #             "IsWaterCollection": trans["iswatercollection"],
    #             "XP": int(trans["xp"]),
    #             "Exists": trans["exists"]
    #         })

    # # Convert to DataFrame
    # unlockable_df = pd.DataFrame(unlockable)
    # if unlockable_df.empty:
    #     print("No transformations can be unlocked.")
    # else:
    #     unlockable_df.sort_values(by=["Base", "Input"], inplace=True)

    # return unlockable_df

get_unlockable_transformations(player.cafecosmos, player.land_id)


[{'base': '20', 'input': '0', 'next': '0', 'yield': '20', 'inputnext': '0', 'yieldquantity': '24', 'unlocktime': '0', 'timeout': '0', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '0', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '23', 'input': '0', 'next': '0', 'yield': '23', 'inputnext': '0', 'yieldquantity': '8', 'unlocktime': '0', 'timeout': '0', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '0', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '41', 'input': '0', 'next': '0', 'yield': '129', 'inputnext': '0', 'yieldquantity': '1', 'unlocktime': '0', 'timeout': '360', 'timeoutyield': '0', 'timeoutyieldquantity': '0', 'timeoutnext': '131', 'isrecipe': False, 'iswatercollection': False, 'xp': '2', 'exists': True}, {'base': '47', 'input': '0', 'next': '0', 'yield': '14', 'inputnext': '0', 'yieldquantity': '3', 'unlocktime': '0', 'timeout': '360', 'timeoutyield': '0', 'timeou

""


In [3]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [7]:
player.place_item(4,0,"bismuth")

Transaction sent: https://explorer.garnetchain.com/tx/0x0a8a83d763374da1e4f4df6334cc7d55845455b8e2f6fde8e99a10dcbc629624


In [4]:
player.display_inventory()

Icon,Item Name,Quantity
,Avocado,8
,Bananas,99
,Coffee,3
,Egg,26
,Raspberry,39
,Sugar,18
,Olives,14
,Bismuth,32
,Crystal,144
,Clay,32


In [7]:
player.get_craftable()

,Item ID,Item Name,Max Craftable,Requirements
0,5,Coffee,45,1 x Coffee Bean
1,27,Glass,144,1 x Crystal
2,29,Blade,8,"4 x Bismuth, 1 x Crystal"
3,82,Cutting Board,44,"1 x Sticks, 1 x Crystal, 3 x Pink Wood"
4,90,Pink Chair,33,4 x Pink Wood
5,92,Green Table,7,"3 x Pink Wood, 2 x Purple Wood"
6,93,Purple Table,3,5 x Purple Wood
7,94,Pink Table,26,5 x Pink Wood
8,96,Fence,31,"5 x Sticks, 2 x Pink Wood"
9,97,Pink Floor,44,3 x Pink Wood


In [4]:
player.cafecosmos.indexer.Transformations.get(input=0)

[{'base': '20',
  'input': '0',
  'next': '0',
  'yield': '20',
  'inputnext': '0',
  'yieldquantity': '24',
  'unlocktime': '0',
  'timeout': '0',
  'timeoutyield': '0',
  'timeoutyieldquantity': '0',
  'timeoutnext': '0',
  'isrecipe': False,
  'iswatercollection': False,
  'xp': '2',
  'exists': True},
 {'base': '23',
  'input': '0',
  'next': '0',
  'yield': '23',
  'inputnext': '0',
  'yieldquantity': '8',
  'unlocktime': '0',
  'timeout': '0',
  'timeoutyield': '0',
  'timeoutyieldquantity': '0',
  'timeoutnext': '0',
  'isrecipe': False,
  'iswatercollection': False,
  'xp': '2',
  'exists': True},
 {'base': '41',
  'input': '0',
  'next': '0',
  'yield': '129',
  'inputnext': '0',
  'yieldquantity': '1',
  'unlocktime': '0',
  'timeout': '360',
  'timeoutyield': '0',
  'timeoutyieldquantity': '0',
  'timeoutnext': '131',
  'isrecipe': False,
  'iswatercollection': False,
  'xp': '2',
  'exists': True},
 {'base': '47',
  'input': '0',
  'next': '0',
  'yield': '14',
  'inputnext

'0x426f99291Ba267E155d9B9B4Fa5d698A23DdE108'

In [15]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [5]:
player.place_item(4,0,"unlock")

Transaction sent: https://explorer.garnetchain.com/tx/0xdac44d1e145ac8f630b8b29702a5e71fa8ed0969b63d4c06a7eba47c722fcd8f
Transaction confirmed in block: 12158899


In [6]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
